In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

import os
import random
import json
import argparse
from time import sleep

from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException, ElementNotInteractableException


In [ ]:
#напишем функцию правильного формата адресса и названия
def get_right_format(name_df, restaurant_name, address_name):
    data = pd.read_csv(name_df)
    data[address_name] = data[address_name].apply(lambda x: str(x))
    data[restaurant_name] = data[restaurant_name].apply(lambda x: str(x))
    return data


def get_parsing_data(path):
    df = get_right_format(path, restaurant_name='Restaurant_name', address_name='address_name')
    
    # Инициализация новых колонок
    new_columns = [
        'restaurant_category', 
        'transport_connection', 
        'Cusiene', 
        'Average_bill', 
        'Price_level' 
    ]
    for col in new_columns:
        if col not in df.columns:
            df[col] = None

    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url='https://yandex.ru/maps/213/moscow/?ll=37.555911%2C55.786351&z=10')
    sleep(1)
    
    for index, row in df[8501:].iterrows():
        # Поиск и ввод запроса
        search_box = driver.find_element(By.CLASS_NAME, 'input__control')
        req = f"{row['Restaurant_name']},{row['Brief_description']}, {row['address_name']}, {row['postcode']}, Москва"
        search_box.send_keys(req)
        search_box.send_keys(Keys.RETURN)
        sleep(2)
        


        try:
            # Ожидаем появления элемента списка ресторанов
            element = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CLASS_NAME, "search-business-snippet-view__head"))
            )
            # print("Найдена страница со списком ресторанов")
            try:
                firts_rest = driver.find_element(By.CLASS_NAME, "search-business-snippet-view__title")
                try:
                    firts_rest.click()
                    sleep(1)
                except (StaleElementReferenceException, ElementNotInteractableException):
                    pass
            except ElementClickInterceptedException:
                try:
                    search_box.send_keys(Keys.COMMAND + 'a')
                    search_box.send_keys(Keys.DELETE)
                    # for i in range(len(req)):
                    #     search_box.send_keys(Keys.BACK_SPACE)
                    search_box = driver.find_element(By.CLASS_NAME, 'input__control')
                    req = f"{row['Restaurant_name']}, {row['address_name']}, Москва"
                    search_box.send_keys(req)
                    search_box.send_keys(Keys.RETURN)
                    element = WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "card-title-view__title-link"))
                    )
                    # print("Найдена карточка ресторана")
                    # print(element)
                except ElementClickInterceptedException:
                    continue
                    # print('fff')
                    
        except TimeoutException:
            try:
                # Если список не найден, ищем карточку ресторана
                element = WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "card-title-view__title-link"))
                )
                # print("Найдена карточка ресторана")
            except TimeoutException:
                # print("Ни один из элементов не найден")
                continue   
        
        
        # Парсинг данных
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Категории заведения
        cat = [i.text for i in soup.find_all('a', class_="business-categories-view__category _outline _clickable")]
        
        
        
        # Транспортная доступность
        transport = [i.text for i in soup.find_all('div', class_="masstransit-stops-view__stop-name")]
        distance = [i.text.replace('\xa0', ' ') for i in soup.find_all('div', class_="masstransit-stops-view__stop-distance-text")]
        tr_dist = dict(zip(transport, distance))
        
        
        
        #Нажмем на кнопку подробнее об организации
        org_info = [i for i in driver.find_elements(By.CLASS_NAME, "card-feature-view__main-content") if i.text == "Подробнее об организации"]
        if len(org_info) > 0:
            try:
                org_info[0].click()
            except ElementClickInterceptedException:
                # print("Не удалось кликнуть на элемент 'Подробнее об организации', элемент перекрыт.")
                continue  # Пропускаем текущую итерацию и переходим к следующей строке
        
        
        
        #Получим уровень цен и средний чек
        percu_html = driver.page_source
        soup_perc = BeautifulSoup(html, 'html.parser')
        
        prices = soup_perc.find_all('div', class_="business-features-view__valued-list")

        # Создаем словарь для данных
        result_dict = {'Кухня': None, 'Средний счёт': None, 'Цены': None}
        if len(prices) > 0:
            # Ищем все элементы с характеристиками
            for item in prices[0].find_all('div', class_="business-features-view__valued"):
                # Извлекаем ключ (название характеристики)
                key = item.find('span', class_="business-features-view__valued-title").text.strip()
                
                # Извлекаем значение
                value = item.find('span', class_="business-features-view__valued-value").text.strip()
                
                # Убираем двоеточие в конце ключа (если есть)
                if key.endswith(':'):
                    key = key[:-1].strip()
                
                result_dict[key] = value

        
        
        
        
        
        # print(f'Название ресторана: {row['Restaurant_name']}')
        # print(f'категории заведения: {cat}')
        # print(f'трансорт: {tr_dist}')
        # print(result_dict)
        # print(f'кухня: {result_dict["Кухня"]}')
        # print(f'средний чек: {result_dict['Средний счёт']}')
        # print(f'уровень цен: {result_dict['Цены']}')
        
        #Запись данных
        df.at[index, 'restaurant_category'] = cat
        df.at[index, 'transport_connection'] = str(tr_dist)
        df.at[index, 'Cusiene'] = result_dict['Кухня']
        df.at[index, 'Average_bill'] = result_dict['Средний счёт']
        df.at[index, 'Price_level'] = result_dict['Цены']
        
        
        # Очистка поиска
        # for i in range(len(req)):
        #     search_box.send_keys(Keys.BACK_SPACE)
        search_box.send_keys(Keys.COMMAND + 'a')
        search_box.send_keys(Keys.DELETE)
        print(index)
        
    driver.quit()
    return df

In [ ]:
path = '2GIS_data-2.csv'
res = get_parsing_data(path)